In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm import tqdm # pip install tqdm
import time
import os

MAIN_URL = 'https://www.house.kg/kupit-kvartiru'

In [29]:
def get_max_pages(url:str)->int:
    "Getting the number of the last page"
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    obj = soup.find("div", class_="listings-wrapper")
    return int(obj.find('nav').find_all('li')[-1].find('a')['data-page'])

In [126]:
def get_url()->None:
  "Loading URLS of each House and saving as file"
  print('___ LOAD URLS ___')

  sub_url = []
  pages = input('How many pages do you want? (For all pages leave blank) ')
  
  if pages == '':
    pages = get_max_pages(MAIN_URL)
  else:
    pages = int(pages)
    
  for page in tqdm(range(1, pages+1)):
    url = f'{MAIN_URL}?page={page}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    objects = soup.find("div", class_="listings-wrapper")
    lincs = objects.find_all('div', class_="left-image")
    for ur in lincs:
      sub_url.append(ur.find('a')['href'])

  print(f'\n ___ Received {len(sub_url)} URLS ___')

  urls = []
  for i in sub_url:
    urls.append(f"https://www.house.kg{i}")

  pd.DataFrame(urls, columns=['URLS']).to_csv('urls.csv', index=False)
  print('\nFILE "urls.csv" SAVED')

In [116]:
get_url()

___ LOAD URLS ___


100%|██████████| 649/649 [21:39<00:00,  2.00s/it]  


 ___ Received 6481 URLS ___

FILE "urls.csv" SAVED


In [24]:
import datetime
current_date = datetime.date.today()

In [176]:
def get_data(ur:str)->dict:
  
  response = requests.get(ur)
  soup = BeautifulSoup(response.text, "html.parser")
  main_content = soup.find('div', class_='main-content')
  header = main_content.find('div', class_='details-header')
  user_info = main_content.find('div', class_='phone-fixable-block')
  main_details = main_content.find('div', class_='details-main')
  atributs = main_details.find_all("div", class_="info-row")
  
  USD_price = header.find("div", class_="price-dollar").text.strip().replace(' ', '').replace('$', '')
  kgs_price  = header.find("div", class_="price-som").text.strip().replace(' ', '').replace('сом', '')
  
  address = header.find("i", class_="fas fa-map-marker-alt")
  number = user_info.find('div', class_='number')
  map_attr = main_details.find('div', id='map2gis')
  latitude = map_attr['data-lat']
  longitude = map_attr['data-lon']
  n_rooms = header.find("div", class_="left").text.strip().split('\n')[0]
  
  describe = main_details.find("div", class_="description")
  added = header.find('span', class_ ='added-span')
  upped = header.find('span', class_ ='upped-span')
  rating = user_info.find('div', class_ ='rating score')
  views = header.find('span', class_ ='view-count')
  atr_dict = {
    'url': ur,
  }
  
  for atr in atributs:
    key = atr.find('div', class_="label").text.strip().replace('\n', ' ')
    val = atr.find('div', class_="info").text.strip().replace('\n', ' ')
    atr_dict[key] = val
    
  try:
    atr_dict['address'] = address.next_sibling.strip()
    atr_dict['n_rooms'] = n_rooms
  except Exception:
    atr_dict['adress'] = np.nan
  try:
    atr_dict['describe'] = describe.find('p').text.strip().replace('\n', ' ')
  except Exception:
    atr_dict['describe'] = np.nan
  try:
    atr_dict['added'] = added.text
    atr_dict['upped'] = upped.text
  except Exception:
    atr_dict['added'] = np.nan
    atr_dict['upped'] = np.nan
  try:
    atr_dict['latitude'] = float(latitude)
    atr_dict['longitude'] = float(longitude)
  except Exception:
    atr_dict['latitude'] = np.nan
    atr_dict['longitude'] = np.nan
  try:
    atr_dict['rating'] = rating.text.strip()
  except Exception:
    atr_dict['rating'] = np.nan
    
  atr_dict['views'] = int(views.text)
  atr_dict['phone_number'] = number.text
  atr_dict['USD_price'] = USD_price
  atr_dict['KGS_price '] = kgs_price
  atr_dict['current_date '] = current_date

  try: 
    hearts = header.find('span', class_ ='favourite-count table-comments').text
    atr_dict['hearts'] = hearts.split(';')[1]
  except Exception:
    atr_dict['hearts'] = np.nan

  return atr_dict

In [175]:
response = requests.get('https://www.house.kg/details/521743066ec024c57fe40-61109241')
soup = BeautifulSoup(response.text, "html.parser")

# main_details = soup.find('div', class_='details-main').find('div', class_='right')
# rating = main_details.find('div', class_='fotorama').find_all('a')

# img_list = []
# for a in rating:
#     img_list.append(a['href'])
    
# img_list

('Добавлено 9 часов назад', ' Поднято 8 часов назад')

In [165]:
data = []
error_urls = []

In [179]:
urls = pd.read_csv('urls.csv').URLS.to_list()

for url in tqdm(urls):
    try:
        data.append(get_data(url))
    except Exception:
        url_index = urls.index(url)
        print('!!!___ERRROR___!!!')
        print(f'With url index: {url_index}')
        print(f'With url: {url}')
        error_urls.append(url)
        
pd.DataFrame(data).to_csv('data_last.csv', index=False)
pd.DataFrame(error_urls).to_csv('error_urls_last.csv', index=False)

  0%|          | 0/6556 [00:00<?, ?it/s]

  1%|          | 43/6556 [00:47<2:26:59,  1.35s/it]

!!!___ERRROR___!!!
With url index: 42
With url: https://www.house.kg/details/728676766ebf0a9b33b32-88503514


  1%|          | 44/6556 [00:49<2:28:26,  1.37s/it]

!!!___ERRROR___!!!
With url index: 43
With url: https://www.house.kg/details/332924866ebf090d3d332-26535346


...

 96%|█████████▌| 6277/6556 [2:16:50<04:28,  1.04it/s]

!!!___ERRROR___!!!
With url index: 6276
With url: https://www.house.kg/details/88505266b2f60d48e3a1-38960698


 98%|█████████▊| 6442/6556 [2:19:48<02:13,  1.17s/it]

!!!___ERRROR___!!!
With url index: 6441
With url: https://www.house.kg/details/195967566a4935d7af6c7-66812990


100%|██████████| 6556/6556 [2:25:25<00:00,  1.33s/it]


In [180]:
data = pd.DataFrame(data)

In [181]:
data

,url,Тип предложения,Серия,Дом,Этаж,Площадь,Отопление,Состояние,Телефон,Санузел,...,current_date,hearts,Интернет,Безопасность,Возможность рассрочки,Возможность обмена,Площадь участка,Канализация,Питьевая вода,Электричество
0,https://www.house.kg/details/515492365e7189104...,от собственника,индивид. планировка,"кирпичный, 2024 г.",1 этаж из 3,35 м2,электрическое,хорошее,возможно подключение,совмещенный,...,2024-09-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.house.kg/details/932931666cc8a829e...,от собственника,элитка,"монолитный, 2022 г.",12 этаж из 15,64 м2,на газе,евроремонт,NaN,2 с/у и более,...,2024-09-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.house.kg/details/266187266e0264c64...,от собственника,индивид. планировка,"кирпичный, 2021 г.",4 этаж из 4,"39 м2, жилая: 13 м2, кухня: 17 м2",на газе,евроремонт,NaN,NaN,...,2024-09-18,NaN,проводной,"видеонаблюдение, ...",NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.house.kg/details/696572651eac369ce...,от собственника,элитка,"монолитный, 2014 г.",4 этаж из 6,"115 м2, кухня: 23 м2",автономное,евроремонт,есть,2 с/у и более,...,2024-09-18,NaN,проводной,"домофон, ...",нет,обмен не предлагать,NaN,NaN,NaN,NaN
4,https://www.house.kg/details/4141411668e6aea34...,от собственника,элитка,"кирпичный, 2021 г.",9 этаж из 9,101 м2,на газе,евроремонт,возможно подключение,2 с/у и более,...,2024-09-18,NaN,adsl,"решетки на окнах, ...",NaN,рассмотрю варианты,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6499,https://www.house.kg/details/911020166ea8db808...,от агента,элитка,"кирпичный, 2022 г.",5 этаж из 5,"79 м2, жилая: 36.1 м2, кухня: 12 м2",центральное,хорошее,нет,совмещенный,...,2024-09-18,NaN,оптика,"домофон, ...",нет,обмен не предлагать,NaN,NaN,NaN,NaN
6500,https://www.house.kg/details/390821266ebbff8a4...,от агента,хрущевка,"кирпичный, 1965 г.",3 этаж из 4,"30 м2, жилая: 18 м2, кухня: 5.5 м2",центральное,среднее,нет,совмещенный,...,2024-09-18,NaN,оптика,домофон,нет,обмен не предлагать,NaN,NaN,NaN,NaN
6501,https://www.house.kg/details/280086366e90b780f...,от собственника,элитка,кирпичный,4 этаж из 12,50 м2,центральное,евроремонт,NaN,NaN,...,2024-09-18,NaN,NaN,NaN,нет,обмен не предлагать,NaN,NaN,NaN,NaN
6502,https://www.house.kg/details/104503366e81ead64...,от собственника,105 серия улучшенная,"кирпичный, 2024 г.",7 этаж из 9,60 м2,центральное,под самоотделку (псо),нет,раздельный,...,2024-09-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [182]:
data.columns

Index(['url', 'Тип предложения', 'Серия', 'Дом', 'Этаж', 'Площадь',
       'Отопление', 'Состояние', 'Телефон', 'Санузел', 'Газ', 'Балкон',
       'Входная дверь', 'Парковка', 'Мебель', 'Пол', 'Высота потолков',
       'Разное', 'Правоустанавливающие документы', 'Возможность ипотеки',
       'address', 'n_rooms', 'describe', 'added', 'upped', 'latitude',
       'longitude', 'rating', 'views', 'phone_number', 'USD_price',
       'KGS_price ', 'current_date ', 'hearts', 'Интернет', 'Безопасность',
       'Возможность рассрочки', 'Возможность обмена', 'Площадь участка',
       'Канализация', 'Питьевая вода', 'Электричество'],
      dtype='object')